In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [62]:
data_age = pd.read_csv('data/cleaned_age.csv')
data_health = pd.read_csv('data/cleaned_health.csv')
data_heat = pd.read_csv('data/cleaned_heat.csv')
data_depr = pd.read_csv('data/cleaned_depr.csv')
data_hhold = pd.read_csv('data/cleaned_hhold.csv')
data_ethgrp = pd.read_csv('data/cleaned_ethgrp.csv')
data_cars = pd.read_csv('data/cleaned_cars.csv')

meta_data = pd.read_csv('data/meta_all.csv')

In [64]:
def prop_of_tot(old_data):
    
    data = old_data.set_index('GEO_CODE').astype(int)
    data['sum'] = data.sum(axis = 1, skipna = all)
    
    new_df = pd.DataFrame(index = data.index, columns = data.columns)
    for i in range(len(data)):
        for j in range(int(len(data.columns) - 1)):
            new_df.iloc[i][j] = (data.iloc[i][j] / data.iloc[i][int(len(data.columns) - 1)])
    
    new_df = new_df.dropna(axis = 1)

    return(new_df)

In [65]:
prop_health = prop_of_tot(data_health)
prop_heat = prop_of_tot(data_heat)
prop_depr = prop_of_tot(data_depr)
prop_cars = prop_of_tot(data_cars)
prop_ethgrp = prop_of_tot(data_ethgrp)

In [67]:
prop_health['badhealth'] = prop_health['F1777'] + prop_health['F1782']
prop_depr['depr_2+'] = prop_depr['F999'] + prop_depr['F1000'] + prop_depr['F1001']
prop_depr['depr_3+'] = prop_depr['F1000'] + prop_depr['F1001']
prop_ethgrp['F166x'] = prop_ethgrp['F1664'] + prop_ethgrp['F1665'] + prop_ethgrp['F1666'] + prop_ethgrp['F1667']

In [68]:
data_ethgrp = data_ethgrp.set_index('GEO_CODE')

for i in range(len(data_ethgrp)):
    for j in range(len(data_ethgrp.columns)):
        data_ethgrp.iloc[i][j] = data_ethgrp.iloc[i][j] ** 2

data_ethgrp['hhi'] = data_ethgrp.sum(axis = 1, skipna = all)

In [69]:
data_ethgrp = data_ethgrp[['hhi']]

In [71]:
# normalise hhi --> turns to array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(data_ethgrp.values)

C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [72]:
hhi_list = []

for i in range(len(x_scaled)):
    hhi_list.append(float(x_scaled[i]))

In [73]:
data_ethgrp['hhi_norm'] = hhi_list

In [74]:
data_ethgrp = data_ethgrp[['hhi_norm']].join(prop_ethgrp[['F166x']])

In [76]:
data_hhold = data_hhold.set_index('GEO_CODE')
data_age = data_age.set_index('GEO_CODE')

In [77]:
prop_hhold = pd.DataFrame(index = data_hhold.index, columns = data_hhold.columns)

for i in range(len(data_hhold)):
    for j in range(1, len(data_hhold.columns)):
        prop_hhold.iloc[i][j] = (data_hhold.iloc[i][j] / data_hhold.iloc[i][0])
        
prop_hhold = prop_hhold.dropna(axis = 1)

In [86]:
# join data

man_data = prop_health.join(prop_heat).join(prop_depr).join(prop_cars).join(prop_hhold).join(data_age).join(data_ethgrp)

In [87]:
tokeep = ['F184', 'F185', 'badhealth', 'F2345', 'F997', 'F1001', 'depr_3+', 'depr_2+', 'F1838', 'F1816', 'F991', 'hhi_norm', 'F166x'] 

man_data = (man_data[tokeep].rename(columns = {'F184':'age_mean',
                                               'F185':'age_median',
                                               'F2345':'centheat',
                                               'F997':'nodepr',
                                               'F1838':'studenthhold',
                                               'F1816':'onefam_hhold',
                                               'F166x':'ethwhite',
                                               'F991':'nocars',
                                               'F1001':'depr_4+',
                                               'hhi_norm':'eth_hhi'})\
           .astype(np.float))

In [89]:
man_data.head()

,age_mean,age_median,badhealth,centheat,nodepr,depr_4+,depr_3+,depr_2+,studenthhold,onefam_hhold,nocars,eth_hhi,ethwhite
GEO_CODE,,,,,,,,,,,,,
E01004766,42.10929,42.0,0.075377,0.937107,0.350943,0.002516,0.051572,0.298113,0.001258,0.510692,0.354717,0.176145,0.863693
E01004767,40.15316,39.0,0.048868,0.958333,0.446615,0.003906,0.046875,0.221354,0.010417,0.558594,0.244792,0.172369,0.796782
E01004768,39.94941,42.0,0.016864,0.991736,0.641322,0.000000,0.008264,0.084298,0.000000,0.796694,0.044628,0.221215,0.937539
E01004769,41.63690,42.5,0.035202,0.982482,0.554745,0.001460,0.016058,0.138686,0.000000,0.645255,0.132847,0.197436,0.930248
E01004770,44.31058,45.0,0.055290,0.978896,0.452922,0.001623,0.022727,0.194805,0.000000,0.733766,0.144481,0.136553,0.822526


In [88]:
man_data.to_csv('data/manchester_data.csv')